<a href="https://colab.research.google.com/github/frason-eclimai/Qwen3-VL-2B-Instruct-experiments-cctv/blob/main/Roboflow_Albumentations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import userdata
ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')
userdata.get('HF_TOKEN')

.

**We install libs and fetch the Roboflow dataset.**

In [1]:
!pip install -q roboflow ultralytics albumentations

from google.colab import userdata
ROBOFLOW_API_KEY = userdata.get("ROBOFLOW_API_KEY")

from roboflow import Roboflow
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

project = rf.workspace("cctv-camera-test").project("my-first-project-brf5z")
version = project.version(6)
dataset = version.download("yolov11")

data_yaml = dataset.location + "/data.yaml"
print("data.yaml:", data_yaml)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 102.1 MB/s eta 0:00:00
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to My-First-Project-6 in yolov11:: 100%|██████████| 532/532 [00:00<00:00, 6616.72it/s]

data.yaml: /content/My-First-Project-6/data.yaml


.

**Define a single CCTV pipeline.
Also fixed few warnings (Albumentations v2 uses different argument names).**

In [2]:
import albumentations as A
from ultralytics.data.augment import Albumentations

custom_aug = A.Compose(
    [
        # camera/viewpoint (safe range)
        A.Affine(
            rotate=(-10, 10),
            translate_percent=(-0.06, 0.06),
            scale=(0.9, 1.1),
            shear=(-6, 6),
            p=0.7
        ),
        A.Perspective(scale=(0.02, 0.07), p=0.25),

        # CCTV realism
        A.MotionBlur(p=0.2),
        A.RandomBrightnessContrast(p=0.35),
        A.RandomGamma(p=0.25),

        # FIXED args for Albumentations v2
        A.GaussNoise(std_range=(0.02, 0.08), p=0.25),
        A.ImageCompression(quality_range=(25, 80), p=0.25),

        # occlusion for crowds/partial people
        A.CoarseDropout(
            num_holes_range=(1, 6),
            hole_height_range=(0.05, 0.18),
            hole_width_range=(0.05, 0.18),
            fill=0,
            p=0.2
        ),
    ],
    bbox_params=A.BboxParams(
        format="yolo",
        label_fields=["class_labels"],
        min_visibility=0.25
    )
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


.

**Train YOLO once (with Albumentations injected)**

In [3]:
from ultralytics import YOLO

def on_train_start(trainer):
    trainer.train_loader.dataset.transforms = Albumentations(custom_aug)

model = YOLO("yolo11n.pt")
model.add_callback("on_train_start", on_train_start)

model.train(
    data=data_yaml,
    epochs=100,
    imgsz=640,
    batch=16,
    name="cam_angle_albu_clean"
)


Ultralytics 8.3.234 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/My-First-Project-6/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=cam_angle_albu_clean, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspectiv

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x786d54654b00>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

.

**Validate and Test**

In [4]:
metrics = model.val(data=data_yaml)
print(metrics)

# quick visual test
pred = model.predict(source=f"{dataset.location}/valid/images", conf=0.25, save=True)
print("Saved preds to:", pred[0].save_dir)

Ultralytics 8.3.234 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1180.7±430.3 MB/s, size: 45.5 KB)
val: Scanning /content/My-First-Project-6/valid/labels.cache... 29 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 29/29 62.3Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 1.5it/s 1.4s
                   all         29        123      0.873      0.805      0.863      0.395
Speed: 8.4ms preprocess, 20.5ms inference, 0.0ms loss, 3.9ms postprocess per image
Results saved to /content/runs/detect/val
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x786d468a8ce0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)'

- **Precision (P) = 0.873:** ~87% of predicted boxes are correct → few false positives.  
- **Recall (R) = 0.805:** ~81% of true objects are detected → ~19% are missed.  
- **mAP@50 = 0.863:** strong detection performance at IoU 0.50.  
- **mAP@50–95 = 0.395:** large drop at stricter IoUs → boxes are generally close, but not tightly localized.


.

**Trying a slightly bigger model**

In [5]:

model = YOLO("yolo11s.pt")
model.train(
    data=data_yaml,
    epochs=100,
    imgsz=832,
    batch=8,
    name="yolo11s_img832"
)


Ultralytics 8.3.234 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/My-First-Project-6/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=832, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11s_img832, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, 

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x786d54635d90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 



.


- **Precision (P) = 0.878:** ~88% of predicted boxes are correct → few false positives.  
- **Recall (R) = 0.879:** ~88% of true objects are detected → ~12% are missed.  
- **mAP@50 = 0.915:** strong detection performance at IoU 0.50.  
- **mAP@50–95 = 0.405:** big drop at stricter IoUs → boxes are close, but not tightly localized.

.

**Training at higher resolution**

In [6]:
model = YOLO("yolo11n.pt")
model.train(
    data=data_yaml,
    epochs=100,
    imgsz=960,
    batch=8,       # lower batch because imgs are bigger
    name="imgsz960"
)

Ultralytics 8.3.234 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/My-First-Project-6/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=imgsz960, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x786d546408f0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

- **Precision (P) = 0.865:** ~86.5% of predicted boxes are correct → relatively few false positives.  
- **Recall (R) = 0.837:** ~83.7% of true objects are detected → about 16.3% are missed.  
- **mAP@50 = 0.893:** strong detection performance at IoU 0.50.  
- **mAP@50–95 = 0.399:** noticeable drop at stricter IoUs → boxes are decent but not tightly localized.


YOLO11s at 832 img outperforms YOLO11n at 960 img on our validation set.

It delivers better precision, recall, and mAP@50, with a slightly higher mAP@50–95.

The n model to 960 resolution wasn’t enough to surpass the larger s model.